In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
##Load Data SEt
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#preprocess the data
## drop the irrelevent column
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
## ENcode ont cat var
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
#ONehpt encode "Geography"
from sklearn.preprocessing import OneHotEncoder
onehotencoder_geo=OneHotEncoder()
geo_encoder=onehotencoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
onehotencoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=onehotencoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Combine all one hot encoded columns
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
##save encode
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehotencoder_geo.pkl','wb') as file:
    pickle.dump(onehotencoder_geo,file)

In [11]:
## Devide the dataset into dependent and independent features
X=data.drop('Exited',axis=1)
y=data['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
X_train

array([[-7.86662014e-01, -1.09609805e+00, -1.16053232e-03, ...,
         9.92527915e-01, -5.73693681e-01, -5.75233311e-01],
       [ 1.28944369e+00,  9.12327143e-01, -7.59059191e-01, ...,
         9.92527915e-01, -5.73693681e-01, -5.75233311e-01],
       [-8.79621971e-01,  9.12327143e-01,  9.46212791e-01, ...,
         9.92527915e-01, -5.73693681e-01, -5.75233311e-01],
       ...,
       [ 1.33075922e+00, -1.09609805e+00, -1.23274585e+00, ...,
         9.92527915e-01, -5.73693681e-01, -5.75233311e-01],
       [ 5.35435149e-01, -1.09609805e+00, -3.80109862e-01, ...,
        -1.00752834e+00, -5.73693681e-01,  1.73842505e+00],
       [ 7.42012830e-01, -1.09609805e+00,  7.56738127e-01, ...,
         9.92527915e-01, -5.73693681e-01, -5.75233311e-01]])

In [13]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

#### ANN Implementation

In [14]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [16]:
(X_train.shape[1],)

(12,)

In [17]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 connectd with inut layer
    Dense(32,activation='relu'),## HL2
    Dense(1,activation='sigmoid')#output layer

])


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss


In [20]:
## Complie the model
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])


In [21]:
## setup the Tensorboard
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping


log_dir="logs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [23]:
#set up early stopping
early_stopping_callbacks=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [24]:
## Train the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_callback,early_stopping_callbacks])

Epoch 1/100


250/250 [==============================] - 17s 42ms/step - loss: 0.4561 - accuracy: 0.7904 - val_loss: 0.3904 - val_accuracy: 0.8330
Epoch 2/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3886 - accuracy: 0.8390 - val_loss: 0.3527 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3573 - accuracy: 0.8550 - val_loss: 0.3355 - val_accuracy: 0.8665
Epoch 4/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3468 - accuracy: 0.8580 - val_loss: 0.3320 - val_accuracy: 0.8660
Epoch 5/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3416 - accuracy: 0.8605 - val_loss: 0.3313 - val_accuracy: 0.8660
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3378 - accuracy: 0.8591 - val_loss: 0.3283 - val_accuracy: 0.8680
Epoch 7/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3343 - accuracy: 0.8612 - val_loss: 0.3286 - val_accuracy:

In [25]:
model.save('model.h5')

f:\Projects\DLProject\dl_env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
#load tenserborad extension
%load_ext tensorboard


In [29]:
%tensorboard --logdir logs/fit20250908-075020

Reusing TensorBoard on port 6007 (pid 16480), started 0:16:22 ago. (Use '!kill 16480' to kill it.)

In [ ]:
#load the pickle file
